In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torchvision import datasets, transforms
import torch.optim as optim

In [3]:
test = pd.read_csv("Airplane_Test.csv")
train = pd.read_csv("Airplane_Train.csv")
test = test.drop("Unnamed: 0",axis=1)
train = train.drop("Unnamed: 0",axis=1)
model_info = [train , test ]
for data in model_info : 
    data['satisfaction'] = data['satisfaction'].map({'neutral or dissatisfied' : 0 ,'satisfied' :1} )
    data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(0)

train_goal_col = train["satisfaction"]
test_goal_col = test["satisfaction"]

train = train.drop( "id", axis='columns' )
test = test.drop( "id", axis='columns' )  
train = train.drop( "Departure/Arrival time convenient", axis='columns' )
test = test.drop( "Departure/Arrival time convenient", axis='columns' ) 
train = train.drop( "Gender", axis='columns' )
test = test.drop( "Gender", axis='columns' ) 
train = train.drop( "satisfaction", axis='columns' )
test = test.drop( "satisfaction", axis='columns' )  


c0 =  train['Departure Delay in Minutes']
# print("max and min in age")
c1 = train['Arrival Delay in Minutes'] 
arrival_departure_delay = []

for i in range(len(c0)) : 
    arrival_departure_delay.append( (int)((c0[i]+c1[i])/2) )
train.insert(20 , 'Arrival/Departure Delay in Minutes' , arrival_departure_delay)

c0 =  test['Departure Delay in Minutes']
# print("max and min in age")
c1 = test['Arrival Delay in Minutes'] 
arrival_departure_delay = []
for i in range(len(c0)) : 
    arrival_departure_delay.append( (int)((c0[i]+c1[i])/2) )
    
test.insert(20 , 'Arrival/Departure Delay in Minutes' , arrival_departure_delay)


train = train.drop( "Departure Delay in Minutes", axis='columns' )
test = test.drop( "Departure Delay in Minutes", axis='columns' )

train = train.drop( "Arrival Delay in Minutes", axis='columns' ) 
test = test.drop( "Arrival Delay in Minutes", axis='columns' ) 


model_info = [train , test ]
for data in model_info : 
    # #map goal column to 1 or 0 
    # data['satisfaction'] = data['satisfaction'].map({'neutral or dissatisfied' : 0 ,'satisfied' :1} )

    # classify customer type
    data['Customer Type'] = data['Customer Type'].map({'Loyal Customer' : 1 ,'disloyal Customer' :0 } )

    # classify type of travel 
    data['Type of Travel'] = data['Type of Travel'].map({'Business travel' : 0 ,'Personal Travel' :1 } )
    # classify Class 
    data['Class'] = data['Class'].map({'Business' : 0 ,'Eco' :1 ,'Eco Plus' :2 } )

    #classify Gender 
    # data['Gender'] = data['Gender'].map({'Male' : 0 ,'Female' :1 } )

    # classify age 
    data.loc[data['Age'] <= 16  , 'Age']                          = 0 
    data.loc[ (data['Age'] > 16 ) & (data['Age'] <= 32 ) , 'Age'] = 1 
    data.loc[ (data['Age'] > 32 ) & (data['Age'] <= 48 ) , 'Age'] = 2 
    data.loc[ (data['Age'] > 48 ) & (data['Age'] <= 64 ) , 'Age'] = 3
    data.loc[ (data['Age'] > 64 ) & (data['Age'] <= 80 ) , 'Age'] = 4
    data.loc[ (data['Age'] > 80 ) , 'Age']                        = 5
    
    # # classify Arrival/Departure Delay in Minutes
    data.loc[  data['Arrival/Departure Delay in Minutes'] == 0.0  ,      'Arrival/Departure Delay in Minutes']                                                      = 0 
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 0.0 ) & (data['Arrival/Departure Delay in Minutes'] <= 10.0 ) , 'Arrival/Departure Delay in Minutes']   = 5 
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 10.0 ) & (data['Arrival/Departure Delay in Minutes'] <= 20.0 ) ,  'Arrival/Departure Delay in Minutes']   = 2
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 20.0 ) & (data['Arrival/Departure Delay in Minutes'] <= 30.0 ) ,  'Arrival/Departure Delay in Minutes']   = 3
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 30.0 ) & (data['Arrival/Departure Delay in Minutes'] <= 50.0 ) ,  'Arrival/Departure Delay in Minutes']   = 3
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 50.0 ) & (data['Arrival/Departure Delay in Minutes'] <= 80.0 ) ,  'Arrival/Departure Delay in Minutes']   = 4 
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 80.0 ) & (data['Arrival/Departure Delay in Minutes'] <= 120.0 ) , 'Arrival/Departure Delay in Minutes']   = 5 
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 120.0 ) &(data['Arrival/Departure Delay in Minutes'] <= 170.0 ) , 'Arrival/Departure Delay in Minutes']   = 6
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 170.0 ) &(data['Arrival/Departure Delay in Minutes'] <= 200.0 ) , 'Arrival/Departure Delay in Minutes']   = 7
    data.loc[ (data['Arrival/Departure Delay in Minutes'] > 200.0 ) , 'Arrival/Departure Delay in Minutes']                                                           = 8
   
    # classify Flight Distance

    data.loc[ (data['Flight Distance'] <= 1000 ) ,  'Flight Distance']                                      = 0
    data.loc[ (data['Flight Distance'] > 1000 ) & (data['Flight Distance'] <= 2000 ) ,  'Flight Distance']  = 1
    data.loc[ (data['Flight Distance'] > 2000 ) &  (data['Flight Distance'] <= 3000 ) , 'Flight Distance']  = 2
    data.loc[ (data['Flight Distance'] > 3000 ) & (data['Flight Distance'] <= 4000 ) , 'Flight Distance']   = 3
    data.loc[ (data['Flight Distance'] > 4000 ) , 'Flight Distance']                                        = 4
    


In [4]:
print( train.shape )
train.describe().transpose()

(59999, 19)


,count,mean,std,min,25%,50%,75%,max
Customer Type,59999.0,0.816964,0.386700,0.0,1.0,1.0,1.0,1.0
Age,59999.0,1.935232,0.988132,0.0,1.0,2.0,3.0,5.0
Type of Travel,59999.0,0.310189,0.462574,0.0,0.0,0.0,1.0,1.0
Class,59999.0,0.592677,0.619505,0.0,0.0,1.0,1.0,2.0
Flight Distance,59999.0,0.711979,0.971078,0.0,0.0,0.0,1.0,4.0
Inflight wifi service,59999.0,2.725629,1.324090,0.0,2.0,3.0,4.0,5.0
Ease of Online booking,59999.0,2.756346,1.396394,0.0,2.0,3.0,4.0,5.0
Gate location,59999.0,2.979733,1.271486,0.0,2.0,3.0,4.0,5.0
Food and drink,59999.0,3.204753,1.327335,0.0,2.0,3.0,4.0,5.0
Online boarding,59999.0,3.251421,1.347590,0.0,2.0,3.0,4.0,5.0


Part 1 : using pytorch 

In [8]:
X_train = torch.tensor(np.array(train) , dtype=torch.float32 ) 
y_train = torch.tensor(train_goal_col ,dtype=torch.float32 ).reshape(-1 , 1 ) 
X_test = torch.tensor(np.array(test) , dtype=torch.float32 ) 
y_test = torch.tensor(test_goal_col ,dtype=torch.float32 ).reshape(-1 , 1 ) 


display(y_train)

tensor([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]])

In [14]:

class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(19 , 50)
        self.act1 = nn.ReLU()
        self.hidden3 = nn.Linear(50 , 100)
        self.act3 = nn.ReLU()
        self.hidden4 = nn.Linear(100 , 100)
        self.act4 = nn.ReLU()  
        self.hidden5 = nn.Linear(100 , 8)
        self.act5 = nn.ReLU()       
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        # x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        x = self.act4(self.hidden4(x))
        x = self.act5(self.hidden5(x))
        # x = self.act6(self.hidden6(x))
        x = self.act_output(self.output(x))
        return x

model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=19, out_features=50, bias=True)
  (act1): ReLU()
  (hidden3): Linear(in_features=50, out_features=100, bias=True)
  (act3): ReLU()
  (hidden4): Linear(in_features=100, out_features=100, bias=True)
  (act4): ReLU()
  (hidden5): Linear(in_features=100, out_features=8, bias=True)
  (act5): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [15]:
class MLP() : 
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    def __init__(self, epoch=100 , batch = 10 , x_train =None, x_test=None , y_train=None , y_test=None ) :
        self.n_epochs = epoch 
        self.batch_size = batch 
        self.x_train = x_train 
        self.x_test = x_test 
        self.y_train = y_train 
        self.y_test = y_test 
    def train_model(self ,model ) :
        for epoch in range(self.n_epochs):
            for i in range(0, len(self.x_train),self.batch_size):
                Xbatch = self.x_train[i:i+self.batch_size]
                y_pred = model(Xbatch)
                ybatch = self.y_train[i:i+self.batch_size]
                loss = self.loss_fn(y_pred, ybatch)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            print(f'Finished epoch {epoch}, latest loss {loss}')

    def Test_Model(self ) : 
        with torch.no_grad():
            y_pred = model(self.x_train)
        accuracy = (y_pred.round() == self.y_train).float().mean()
        print(f"Train Accuracy {accuracy}")

        with torch.no_grad():
            y_pred = model(self.x_test)
        accuracy = (y_pred.round() == self.y_test).float().mean()
        print(f"Test Accuracy {accuracy}")

In [16]:
mlp = MLP( epoch=100 , batch=250 , x_train=X_train , x_test=X_test , y_train=y_train , y_test=y_test )
mlp.train_model(model=model)
mlp.Test_Model() 

Finished epoch 0, latest loss 0.3036256730556488
Finished epoch 1, latest loss 0.212123304605484
Finished epoch 2, latest loss 0.1887224316596985
Finished epoch 3, latest loss 0.1775503307580948
Finished epoch 4, latest loss 0.1750360131263733
Finished epoch 5, latest loss 0.17374643683433533
Finished epoch 6, latest loss 0.17107805609703064
Finished epoch 7, latest loss 0.17503340542316437
Finished epoch 8, latest loss 0.17152415215969086
Finished epoch 9, latest loss 0.16501815617084503
Finished epoch 10, latest loss 0.16576433181762695
Finished epoch 11, latest loss 0.16354767978191376
Finished epoch 12, latest loss 0.16219323873519897
Finished epoch 13, latest loss 0.16277649998664856
Finished epoch 14, latest loss 0.16199126839637756
Finished epoch 15, latest loss 0.16345013678073883
Finished epoch 16, latest loss 0.16361577808856964
Finished epoch 17, latest loss 0.16071374714374542
Finished epoch 18, latest loss 0.1610446274280548
Finished epoch 19, latest loss 0.160127907991409